In [1]:
## roadTrip.ipynb -- visiting each city (and pulling all the data)

to make CSV file:
* go to lennar, xpath: /html/body/div[1]/div/main/div/div[2]/footer/div[1]/div[1]/div[3]/div/div[2]/div/div//a/@href

In [2]:
import pandas as pd
import os

In [3]:
cities = pd.read_csv('./cities-06-2023.csv')

In [4]:
cities

,url
0,/new-homes/alabama/coastal-alabama
1,/new-homes/alabama/huntsville
2,/new-homes/arizona/phoenix
3,/new-homes/arizona/tucson
4,/new-homes/california/central-california
...,...
64,/new-homes/texas/houston
65,/new-homes/texas/san-antonio
66,/new-homes/texas/texas-hill-country
67,/new-homes/utah/salt-lake-city


In [5]:
cities['state'] = cities['url'].str.extract(r'/new-homes/([^/]+)/')
cities['market'] = cities['url'].str.extract(r'/new-homes/[^/]+/([^/]+)')

In [6]:
cities['url'] = "https://www.lennar.com"+ cities['url']

In [7]:
cities

,url,state,market
0,https://www.lennar.com/new-homes/alabama/coast...,alabama,coastal-alabama
1,https://www.lennar.com/new-homes/alabama/hunts...,alabama,huntsville
2,https://www.lennar.com/new-homes/arizona/phoenix,arizona,phoenix
3,https://www.lennar.com/new-homes/arizona/tucson,arizona,tucson
4,https://www.lennar.com/new-homes/california/ce...,california,central-california
...,...,...,...
64,https://www.lennar.com/new-homes/texas/houston,texas,houston
65,https://www.lennar.com/new-homes/texas/san-ant...,texas,san-antonio
66,https://www.lennar.com/new-homes/texas/texas-h...,texas,texas-hill-country
67,https://www.lennar.com/new-homes/utah/salt-lak...,utah,salt-lake-city


In [8]:
from tools import *

In [9]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [10]:
# ## pseudo code for pulling all of the data

# # set up
# # SER = Service('/Users/jimmydelano/Downloads/chromedriver')
# # OP = webdriver.ChromeOptions()

# # open selenium driver with first URL
# # driver = webdriver.Chrome(service=SER, options=OP)
# s=Service(ChromeDriverManager().install())    
# driver = webdriver.Chrome(service=s)
# driver.maximize_window()

# for u in cities['url'][0:]:
#     state, city = u.split('/')[-2], u.split('/')[-1]
#     print(f"Scraping data for {city}, {state}")
    
#     driver.get(u)
#     time.sleep(15)
    
#     closeCookies(driver)
#     print("huh")
#     time.sleep(5)
#     closeCookies(driver)
    
#     total = driver.find_element(By.XPATH, '//*[@id="main"]/div/div[1]/div[2]/div/div[1]/div/div[4]/div[1]/div/div[1]').text.splitlines()[0]
#     assert 'home' in total
#     total = int(total.split('home')[0])
#     print(f"\t Found {total} homes")
#     time.sleep(3)
    
#     loadMoreHomes(driver)
    
#     df = scrapeItUp(driver)
#     assert total == df.shape[0], "Problem scraping. Fuck me"
#     # print(f'{total}. Done!', end='')
    
#     df_clean = cleanItUp(df)

#     outdir = f'./data-06-2023/{state}'
#     if not os.path.exists(outdir):
#         os.mkdir(outdir)

#     df_clean.to_csv(f'./data-06-2023/{state}/{city}.csv', index = False)
#     time.sleep(5)
    

In [15]:
s=Service(ChromeDriverManager().install())    
driver = webdriver.Chrome(service=s)
driver.maximize_window()

for u in cities['url'][61:]:
    state, city = u.split('/')[-2], u.split('/')[-1]
    print(f"Scraping data for {city}, {state}")

    driver.get(u)
    time.sleep(15)

    closeCookies(driver)

    total = driver.find_element(By.XPATH, '//*[@id="main"]/div/div[1]/div[2]/div[1]/div[2]/div/div[4]/div[1]/div/div[1]/button[1]').text.splitlines()[0]

    assert 'home' in total
    total = int(total.split('home')[0])
    print(f"\t Found {total} homes")
    time.sleep(3)

    driver.find_element(By.XPATH, '//*[@id="main"]/div/div[1]/div[2]/div[1]/div[2]/div/div[4]/div[1]/div/div[1]/button[1]').click()
    loadMoreHomes(driver)

    df = scrapeItUp(driver)
    assert total == df.shape[0], "Problem scraping. Fuck me"
        # print(f'{total}. Done!', end='')

    df_clean = cleanItUp(df)

    outdir = f'./data-06-2023/{state}'
    if not os.path.exists(outdir):
        os.mkdir(outdir)

    df_clean.to_csv(f'./data-06-2023/{state}/{city}.csv', index = False)
    time.sleep(5)

Scraping data for austin-central-texas, texas
	 Closed cookies window. Continue...
	 Found 556 homes
	 Loading homes.......................................
	 Scraping.......................................................556. Done!
Scraping data for corpus-christi, texas
	 No cookies window. Continue...
	 Found 6 homes
	 Loading homes
	 Scraping6. Done!
Scraping data for dallas-ft-worth, texas
	 No cookies window. Continue...
	 Found 397 homes
	 Loading homes...................
	 Scraping.......................................397. Done!
Scraping data for houston, texas
	 No cookies window. Continue...
	 Found 571 homes
	 Loading homes...............................
	 Scraping.........................................................571. Done!
Scraping data for san-antonio, texas
	 No cookies window. Continue...
	 Found 457 homes
	 Loading homes........................
	 Scraping.............................................457. Done!
Scraping data for texas-hill-country, texas
	 No cooki

In [ ]:
# driver.find_element(By.XPATH, f'//*[@id="main"]/div/div[1]/div[2]/div/div[3]/div/div[2]/div[1]').text.splitlines()

In [17]:
cities['url']

0     https://www.lennar.com/new-homes/alabama/coast...
1     https://www.lennar.com/new-homes/alabama/hunts...
2      https://www.lennar.com/new-homes/arizona/phoenix
3       https://www.lennar.com/new-homes/arizona/tucson
4     https://www.lennar.com/new-homes/california/ce...
                            ...                        
64       https://www.lennar.com/new-homes/texas/houston
65    https://www.lennar.com/new-homes/texas/san-ant...
66    https://www.lennar.com/new-homes/texas/texas-h...
67    https://www.lennar.com/new-homes/utah/salt-lak...
68    https://www.lennar.com/new-homes/virginia/rich...
Name: url, Length: 69, dtype: object